## 라이브러리 로드

In [ ]:
import os
import sys

import boto3
import polars as pl
from dotenv import load_dotenv

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%matplotlib inline

import seaborn as sns

sns.set_style("whitegrid")

load_dotenv()

In [ ]:
# 한글 흐림현상 방지
%config InlineBackend.figure_format = 'retina'

font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"
font = fm.FontProperties(fname=font_path).get_name()
plt.rc("font", family=font)


matplotlib.rcParams["axes.unicode_minus"] = False

## 데이터 로드

In [ ]:
s3 = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name="ap-northeast-2",
)

BUCKET_NAME = os.getenv("BUCKET_NAME")
PREP_PATH = os.getenv("PREP_PATH")
TABLE_NAME = os.getenv("TABLE_NAME")

## 데이터 전처리

In [ ]:
for month in ["07", "08", "09", "10", "11", "12"]:
    s3.download_file(
        BUCKET_NAME,
        os.path.join(PREP_PATH, "2023", month, f"{TABLE_NAME}.csv"),
        os.path.join("data", f"{TABLE_NAME}_2023{month}.csv"),
    )

    df = pl.read_csv(os.path.join("data", f"{TABLE_NAME}_2023{month}.csv"))

    df = df.with_columns(pl.lit(2023).alias("YEAR"), pl.lit(int(month)).alias("MONTH"))
    df = df.with_columns(pl.all().map_elements(lambda x: None if x == "null" else x))

    df.write_csv(os.path.join("prep", f"{TABLE_NAME}_2023{month}.csv"))

In [ ]:
df